In [16]:
import os
from collections import OrderedDict
os.chdir('/home/ruidong/Documents/Research/share/ID_conversion/liver/')

In [17]:
database = '/home/ruidong/Documents/Research/Cancer_Genomics/Data/Databases/Gene_ID_conversion/'

entrez2ensembl = OrderedDict()
with open(database+'Ensembl_HGNC_Entrez_gene_id.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if len(lst) == 4:
            entrez2ensembl[lst[-1]] = lst[0]
        

In [18]:
ensembl2symbol = OrderedDict()
with open(database+'Homo_sapiens.GRCh38.86.gff3.clean.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        ensemblName = lst[0].split('.')[0]
        if ensemblName in entrez2ensembl.values():
            ensembl2symbol[ensemblName] = lst[1]+'\t'+lst[2]

In [26]:
fh = open('TCGA_ID_converted.txt', 'wt')
fh.write('\t'.join(['gene_id','gene_type', 'protein_coding=0', 'non_coding=1', 'new_id'])+'\n')
with open('TCGA_ID.txt') as f:
     for line in f:
        line = line.rstrip()
        entrezID = line.split('|')[1]
        if entrezID in entrez2ensembl.keys() and entrez2ensembl[entrezID] in ensembl2symbol.keys():
            geneInfo = ensembl2symbol[entrez2ensembl[entrezID]].split('\t')
            if geneInfo[1] == 'protein_coding':
                if line.split('|')[0] != geneInfo[0]:
                    fh.write('\t'.join([line,geneInfo[1],'0','',geneInfo[0]])+'\n')
                else:
                    fh.write('\t'.join([line,geneInfo[1],'0','',''])+'\n')
                    
            else:
                if line.split('|')[0] != geneInfo[0]:
                    fh.write('\t'.join([line,geneInfo[1],'','1',geneInfo[0]])+'\n')
                else:
                    fh.write('\t'.join([line,geneInfo[1],'','1',''])+'\n')
                    
fh.close()

In [1]:
######################################################
######## Step0. Set up Environment Variables #########
######################################################

import re, os, scipy, json
import numpy as np
from scipy import stats
from os import system
from collections import OrderedDict, defaultdict
from operator import itemgetter
from pprint import pprint
from xml.etree.ElementTree import parse


project = '/home/ruidong/Documents/Research/Cancer_Genomics/Data/Liver/TCGA-LIHC/'
databases = '/home/ruidong/Documents/Research/Cancer_Genomics/Data/Databases/'

RNASeq = 'Gene_Expression_Quantification/'
miRNAs = 'miRNA_Expression_Quantification/'
isoform = 'Isoform_Expression_Quantification/'
methylation = 'Methylation/'
CNS = 'Copy_Number_Segment/'
maskedCNS = 'Masked_Copy_Number_Segment/'
miRNAmRNA = 'miRNA_mRNA_network/'
survival = 'survival/'
clinical = 'Clinical/'
biospecimen = 'Biospecimen/'


downloads = 'Downloads/'
clean = 'Clean/'
analysis = 'Analysis/'


os.chdir(project)

In [2]:
########################################
##### GRCh38.86


proteinCodingSymbol = OrderedDict()
longNonCodingSymbol = OrderedDict()
otherNonCodingSymbol = OrderedDict()
ensemblToSymbol= OrderedDict()

lnc = ['3prime_overlapping_ncrna', 'ambiguous_orf', 'antisense', 'lincRNA', 'ncrna_host', 
       'non_coding', 'processed_transcript', 'retained_intron', 'sense_intronic', 'sense_overlapping']

with open(databases+'Gene_ID_conversion/Homo_sapiens.GRCh38.86.gff3.clean.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        geneID = lst[0].split('.')[0]
        if lst[2] == 'protein_coding':
            proteinCodingSymbol[geneID] = lst[1]+'\t'+lst[2]
            ensemblToSymbol[geneID] = '\t'.join(lst[1:4])
        elif lst[2] in lnc:
            longNonCodingSymbol[geneID] = lst[1]+'\t'+lst[2]
            ensemblToSymbol[geneID] = '\t'.join(lst[1:4])
        else:
            otherNonCodingSymbol[geneID] = lst[1]+'\t'+lst[2]
            ensemblToSymbol[geneID] = '\t'.join(lst[1:4])

In [3]:
ensemblToSymbol['ENSG00000237613']

'FAM138A\tlincRNA\tfamily with sequence similarity 138 member A [Source:HGNC Symbol%3BAcc:HGNC:32334]'

In [12]:
########################################
##### GRCh38.86

proteinCodingSymbol = OrderedDict()
longNonCodingSymbol = OrderedDict()
otherNonCodingSymbol = OrderedDict()
ensemblToSymbol= OrderedDict()

fh = open('Homo_sapiens.GRCh38.86.biotype.txt', 'wt')

lnc = ['3prime_overlapping_ncrna', 'ambiguous_orf', 'antisense', 'lincRNA', 'ncrna_host', 
       'non_coding', 'processed_transcript', 'retained_intron', 'sense_intronic', 'sense_overlapping']

pseudogene = ['disrupted_domain', 'IG_C_pseudogene', 'IG_J_pseudogene', 'IG_pseudogene', 'IG_V_pseudogene', 
              'processed_pseudogene', 'pseudogene', 'transcribed_processed_pseudogene', 
              'transcribed_unprocessed_pseudogene', 'translated_processed_pseudogene', 
              'translated_unprocessed_pseudogene', 'TR_J_pseudogene', 'TR_V_pseudogene', 'unitary_pseudogene', 
              'unprocessed_pseudogene']

protein = ['IG_C_gene', 'IG_D_gene', 'IG_J_gene', 'IG_LV_gene', 'IG_M_gene', 'IG_V_gene', 'IG_Z_gene', 
           'nonsense_mediated_decay', 'nontranslating_CDS', 'non_stop_decay', 'polymorphic_pseudogene', 
           'protein_coding', 'TR_C_gene', 'TR_D_gene', 'TR_gene', 'TR_J_gene', 'TR_V_gene']

snc = ['miRNA', 'miRNA_pseudogene', 'misc_RNA', 'misc_RNA_pseudogene', 'Mt_rRNA', 'Mt_tRNA', 'Mt_tRNA_pseudogene', 
       'ncRNA', 'pre_miRNA', 'RNase_MRP_RNA', 'RNase_P_RNA', 'rRNA', 'rRNA_pseudogene', 'scRNA_pseudogene', 
       'snlRNA', 'snoRNA', 'snoRNA_pseudogene', 'snRNA', 'snRNA_pseudogene', 'SRP_RNA', 'tmRNA', 'tRNA', 
       'tRNA_pseudogene']

with open(databases+'Gene_ID_conversion/Homo_sapiens.GRCh38.86.gff3.clean.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        geneID = lst[0].split('.')[0]
        if lst[2] in lnc:
            fh.write(lst[0]+'\t'+lst[1]+'\t'+'long_non_coding\n')
        elif lst[2] in pseudogene:
            fh.write(lst[0]+'\t'+lst[1]+'\t'+'pseudogene\n')
        elif lst[2] in protein:
            fh.write(lst[0]+'\t'+lst[1]+'\t'+'protein_coding\n')
        elif lst[2] in snc:
            fh.write(lst[0]+'\t'+lst[1]+'\t'+'small_non_coding\n')
        else:
            fh.write(lst[0]+'\t'+lst[1]+'\t'+lst[2]+'\n')
            
fh.close()

In [10]:
check={}
with open(databases+'Gene_ID_conversion/Homo_sapiens.GRCh38.86.gff3.clean.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[2] not in check.keys():
            print (lst[2])
            check[lst[2]]=1

biotype
transcribed_unprocessed_pseudogene
unprocessed_pseudogene
miRNA
lincRNA
protein_coding
processed_pseudogene
antisense
processed_transcript
snRNA
transcribed_processed_pseudogene
sense_intronic
misc_RNA
TEC
unitary_pseudogene
snoRNA
scaRNA
rRNA
3prime_overlapping_ncRNA
polymorphic_pseudogene
transcribed_unitary_pseudogene
sense_overlapping
pseudogene
IG_V_pseudogene
bidirectional_promoter_lncRNA
ribozyme
sRNA
non_coding
macro_lncRNA
TR_V_gene
TR_V_pseudogene
TR_C_gene
TR_D_gene
TR_J_gene
TR_J_pseudogene
IG_C_gene
IG_C_pseudogene
IG_J_gene
IG_J_pseudogene
IG_D_gene
IG_V_gene
IG_pseudogene
scRNA
vaultRNA
Mt_tRNA
Mt_rRNA


In [42]:
################################################################################
################### Step4. Prepare input for downstream analysis ###############
################################################################################

os.chdir(project+RNASeq)

fh1 = open(analysis+'LIHC.RNAseq.counts.all.txt', 'wt')
fh2 = open(analysis+'LIHC.RNAseq.counts.long_non_coding.txt', 'wt')

with open(analysis+'Merged_RNAseq_counts_data_TCGA_LIHC.txt', 'rt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == '':
            fh1.write(line+'\n')
            fh2.write(line+'\n')

        elif lst[0].startswith('ENSG'):
            ensemblID = lst[0].split('.')[0]
            if ensemblID  in ensemblToSymbol.keys():
                fh1.write(ensemblToSymbol[ensemblID].split('\t')[0]+'\t'+'\t'.join(lst[1:])+'\n')
                if ensemblID  in longNonCodingSymbol.keys():
                    fh2.write(longNonCodingSymbol[ensemblID].split('\t')[0]+'\t'+'\t'.join(lst[1:])+'\n')
                    
                    
fh1.close()
fh2.close()

In [41]:
check = {}
with open(analysis+'LIHC.RNAseq.counts.long_non_coding.txt', 'rt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0] not in check.keys():
            check[lst[0]] = 0
        check[lst[0]] += 1
        
        
for ke,val in check.items():
    if val > 1:
        print (ke)
        print (val)

LINC01598
2
PGM5-AS1
2
LINC00864
2
FAM95B1
2
PROX1-AS1
2
LINC01505
2
LINC01422
2
LINC00484
2
PRICKLE2-AS1
2
LINC01297
2
RAET1E-AS1
2
DNAJC9-AS1
2
LINC01481
2
PABPC1L2B-AS1
2


In [10]:
##########################################################
############### Add gene type
os.chdir('/home/ruidong/Documents/Research/share/ID_conversion/liver/')

ensemblType = OrderedDict()
symbolType = OrderedDict()

with open('Homo_sapiens.GRCh38.86.biotype.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0] == 'geneID':
            continue
        
        ensemblType[lst[0].split('.')[0]] = lst[-1]
        symbolType[lst[1]] = lst[-1]
        
        
fh = open('LIHC.RNAseq.counts.all.biotype.normalized.txt', 'wt')
with open('LIHC.RNAseq.counts.all.normalized.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if line.startswith('\t'):
            fh.write('\t'+'\t'.join(['biotype']+lst[1:])+'\n')
        else:
            fh.write('\t'.join([lst[0],symbolType[lst[0]]]+lst[1:])+'\n')
                     
                     
fh.close()
        
        
                     
                     
fh = open('Merged_RNAseq_counts_data_TCGA_LIHC.biotype.normalized.txt', 'wt')
with open('Merged_RNAseq_counts_data_TCGA_LIHC.normalized.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if line.startswith('\t'):
            fh.write('\t'+'\t'.join(['biotype']+lst[1:])+'\n')
        elif lst[0].split('.')[0] in ensemblType.keys():
            fh.write('\t'.join([lst[0],ensemblType[lst[0].split('.')[0]]]+lst[1:])+'\n')
        else:
            fh.write('\t'.join([lst[0],'unknown']+lst[1:])+'\n')
                     
fh.close()
        
    

In [12]:
import os
from collections import OrderedDict
os.chdir('/home/ruidong/Documents/Research/share/ID_conversion/liver/')



v79 = OrderedDict()
v86 = OrderedDict()
with open('Homo_sapiens.GRCh38.79.clean.gtf') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if line.startswith('ENSG'):
            gene = lst[0].split('.')[0]
            v79[gene] = '\t'.join([lst[i] for i in range(5) if i != 3])
            
            
with open('Homo_sapiens.GRCh38.86.gff3.clean.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if line.startswith('ENSG'):
            gene = lst[0].split('.')[0]
            v86[gene] = line
            

            
fh1 = open('TCGA.Homo_sapiens.GRCh38.79.biotype.txt', 'wt')
fh2 = open('TCGA.Homo_sapiens.GRCh38.86.biotype.txt', 'wt')
fh3 = open('TCGA.Homo_sapiens.GRCh38.79.diff.86.biotype.txt', 'wt')
fh4 = open('TCGA.Homo_sapiens.GRCh38.79.not.86.biotype.txt', 'wt')




fh1.write('\t'.join(['TCGA','GRCh38.79','geneSymbol','biotype','description'])+'\n')
fh2.write('\t'.join(['TCGA','GRCh38.86','geneSymbol','biotype','description'])+'\n')
fh3.write('\t'.join(['TCGA','GRCh38.79','geneSymbol','biotype','description',
                     'GRCh38.86','geneSymbol','biotype','description'])+'\n')

fh4.write('\t'.join(['TCGA','GRCh38.79','geneSymbol','biotype','description'])+'\n')

with open('RNAseq.counts_only.TCGA-LIHC.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if line.startswith('ENSG'):
            gene = lst[0].split('.')[0]
            if gene in v79.keys():
                fh1.write(lst[0]+'\t'+v79[gene]+'\n')
            else:
                fh1.write(lst[0]+'\t'+'\t'.join(['NA']*4)+'\n')
                
            if gene in v86.keys():
                fh2.write(lst[0]+'\t'+v86[gene]+'\n')
            else:
                fh2.write(lst[0]+'\t'+'\t'.join(['NA']*4)+'\n')
                
            if gene in v79.keys() and gene in v86.keys():
                if v79[gene].split('\t')[1] != v86[gene].split('\t')[1]:
                    fh3.write(lst[0]+'\t'+v79[gene]+'\t'+v86[gene]+'\n')
                    
            if gene in v79.keys() and gene not in v86.keys():
                fh4.write(lst[0]+'\t'+v79[gene]+'\n')
                    
                    

fh1.close()
fh2.close()
fh3.close()
fh4.close()